# Census Data

In [392]:
import cenpy
import pandas as pd
import geopandas as gpd
acs = cenpy.remote.APIConnection("ACSDT5Y2015")

In [393]:
censusRace = acs.query(
    cols=["NAME","B02001_001E","B02001_002E"],
    geo_unit="block group:*",
    geo_filter={
                "state" : "42", 
                "county" : "101"
               },
).rename(columns={"B02001_001E": "totalPop","B02001_002E": "whitePop"}, errors="raise")

In [394]:
acs.set_mapservice("tigerWMS_ACS2015")
where_clause = "STATE = 42 AND COUNTY = 101"
censusTract = acs.mapservice.layers[10].query(where=where_clause)
censusTract = censusTract[["STATE","COUNTY","TRACT",'BLKGRP',"geometry"]]
censusTract = censusTract.to_crs(4326)

KeyboardInterrupt: 

In [ ]:
censusTract.shape

In [ ]:
censusRace2 = censusRace
censusRace2['whitePop'] = pd.to_numeric(censusRace2['whitePop'], errors='ignore')
censusRace2['totalPop'] = pd.to_numeric(censusRace2['totalPop'], errors='ignore')
censusRace2['whitePercep'] = censusRace2['whitePop'] / censusRace2['totalPop'] 
censusRace2['nonwhitePercep'] = 1- censusRace2['whitePercep']
censusRace2.head()

In [ ]:
censusRace2.shape

In [ ]:
censusRaceWithGeometry = censusTract.merge(
    censusRace2,
    left_on=["STATE", "COUNTY", "TRACT",'BLKGRP'],
    right_on=["state", "county", "tract",'block group'])
censusRaceWithGeometry = censusRaceWithGeometry.drop(columns=["STATE","COUNTY","TRACT","BLKGRP"])

In [ ]:
censusRaceWithGeometry.head()

In [ ]:
censusRaceWithGeometry.to_file('censusRaceWithGeometry.geojson', driver="GeoJSON") 

# 311 Complaints

In [ ]:
complaint311 = pd.read_csv ("downloaded_public_cases_fc.csv")

In [ ]:
ndService=['Abandoned Vehicle','Graffiti Removal','Vacant House or Commercial','Street Light Outage','Traffic Signal Emergency','Dead Animal in Street', 'Street Defect','Vacant Lot Clean-Up', 'Alley Light Outage', 'Street Trees', 'Abandoned Bike',
       'Complaint (Streets)', 'Street Paving', 'Tree Dangerous']

In [ ]:
ndService

In [ ]:
complaint311Filter = complaint311.query('service_name in @ndService')
complaint311Filter = complaint311Filter[["service_request_id","service_name","agency_responsible","lat","lon"]].dropna()

In [ ]:
complaint311Filter['geometry'] = gpd.points_from_xy(complaint311Filter['lon'], complaint311Filter['lat'])
complaint311Filter = gpd.GeoDataFrame(complaint311Filter, geometry='geometry', crs="EPSG:4326")

In [ ]:
complaint311Filter

# Crime

In [395]:
import carto2gpd
import matplotlib
import numpy as np
from sklearn.neighbors import NearestNeighbors
import altair as alt
from vega_datasets import data
from altair import datum

In [396]:
url = "https://phl.carto.com/api/v2/sql"
table_name = "incidents_part1_part2"
where = "dispatch_date >= '2015-01-01' AND dispatch_date < '2016-01-01'"
crime = carto2gpd.get(url, table_name, where=where)

In [397]:
crime = crime.drop(columns=["objectid","dc_dist","psa","dc_key","ucr_general","point_x","point_y"])
crime["count"]=1
crime = crime.rename(columns={"text_general_code": "crimeType"}, errors="raise")

In [398]:
total = crime.groupby("crimeType").sum("count").sort_values(by="count",ascending=False)
total = total.reset_index()

alt.Chart(total).mark_bar().encode(
    alt.X('crimeType:N',axis=alt.Axis(labelAngle=90),sort='-y'),
    alt.Y('count:Q'),
    color = alt.value("#F79D46"))

In [399]:
selectedCrimeType = total[0:20].crimeType
selectedCrimeType
selectCrime = crime[crime['crimeType'].isin(selectedCrimeType)]

In [400]:
def get_xy_from_geometry(df): 
    x = df.geometry.centroid.x
    y = df.geometry.centroid.y
    return np.column_stack((x, y))

In [401]:
pointsWithSeg = gpd.read_file("pointsWithSeg.geojson")

pointsWithSegXY = get_xy_from_geometry(pointsWithSeg.to_crs(epsg=3857))
crimeXY = get_xy_from_geometry(selectCrime.to_crs(epsg=3857))

nbrs = NearestNeighbors(n_neighbors=1)
nbrs.fit(pointsWithSegXY)

Dists, Indices = nbrs.kneighbors(crimeXY)

In [402]:
selectCrime["nearestSVPoint"] = Indices
selectCrime2 = selectCrime[['geometry','cartodb_id','crimeType','count','nearestSVPoint']].merge(pointsWithSeg.drop(columns=["geometry","fid","SEG_ID"]),how="left",left_on="nearestSVPoint",right_index=True)
selectCrime2 = selectCrime2.drop(columns="nearestSVPoint").rename(columns={"pointId": "nearestSVPoint"}, errors="raise")

/Users/zh3n/opt/miniconda3/envs/musa550/lib/python3.8/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [403]:
# selectCrime2 = round(selectCrime2,3)
selectCrime2["crimeTypeValue"] = selectCrime2.crimeType.rank(method="first",pct=True)
selectCrime2 = selectCrime2.rename(columns={"crimeTypeValue": "All"}, errors="raise")

In [ ]:
# selectCrime2.to_csv('selectCrime2.csv') 

In [404]:
sampleCrime = selectCrime2.groupby('crimeType', group_keys=False).apply(lambda x: x.sample(min(len(x), 200)))
alt.data_transformers.enable('json')

selection = alt.selection_multi(fields=['crimeType'], bind='legend')

chart=alt.Chart(sampleCrime).transform_window(
    index='count()'
).transform_fold(
    ['All','wall', 'lives','building','infrastructure','road','sidewalk','sky','green','transportation']
).transform_joinaggregate(
     min='min(value)',
     max='max(value)',
     groupby=['key']
).transform_calculate(
    minmax_value=(datum.value-datum.min)/(datum.max-datum.min)
).mark_line(size=0.5).encode(
    x=alt.X('key:N',axis=alt.Axis(ticks=False, domain=False,labelAngle=0)),
    y=alt.Y('minmax_value:Q',axis=alt.Axis(ticks=False, domain=False)),
    color=alt.condition(selection,
                        alt.Color('crimeType:N', scale=alt.Scale(scheme='category20b')),
                        alt.value('lightgray')),   
    opacity=alt.condition(selection, alt.value(1), alt.value(0.07)),   
    detail='index:N',
).properties(
    width=1000,
    height=500,
    title="Relationship between street component and crime type"
).configure_view(
    strokeWidth=0
).add_selection(selection).interactive()

chart.encoding.x.title = 'Street Component'
chart.encoding.y.title = 'Normalized porportion'
chart

alt.Chart(...)

In [405]:
print(
chart.to_json()
)

{'config': {'view': {'continuousWidth': 400,
   'continuousHeight': 300,
   'strokeWidth': 0}},
 'data': {'url': 'altair-data-0c0567b987f817e57ac09b48bf160f14.json',
  'format': {'type': 'json'}},
 'mark': {'type': 'line', 'size': 0.5},
 'encoding': {'color': {'condition': {'type': 'nominal',
    'field': 'crimeType',
    'scale': {'scheme': 'category20b'},
    'selection': 'selector115'},
   'value': 'lightgray'},
  'detail': {'type': 'nominal', 'field': 'index'},
  'opacity': {'condition': {'value': 1, 'selection': 'selector115'},
   'value': 0.07},
  'x': {'type': 'nominal',
   'axis': {'domain': False, 'labelAngle': 0, 'ticks': False},
   'field': 'key',
   'title': 'Street Component'},
  'y': {'type': 'quantitative',
   'axis': {'domain': False, 'ticks': False},
   'field': 'minmax_value',
   'title': 'Normalized porportion'}},
 'height': 500,
 'selection': {'selector115': {'type': 'multi',
   'fields': ['crimeType'],
   'bind': 'legend'},
  'selector116': {'type': 'interval',
   